Table 4.1.1: Substance breakdown of all clients in treatment

In [1]:
from gssutils import *

if is_interactive():
    scraper = Scraper('https://www.gov.uk/government/collections/alcohol-and-drug-misuse-and-treatment-statistics')
    scraper.select_dataset(title=lambda x: x.startswith('Substance misuse treatment for adults'), latest=True)
    tabs = {tab.name: tab for tab in scraper.distribution(title=lambda x: x.startswith('Data tables')).as_databaker()}

In [2]:
tab = tabs['Table 4.1.1']

https://assets.publishing.service.gov.uk/government/uploads/system/uploads/attachment_data/file/752515/AdultSubstanceMisuseNDTMSDataTables2017-18.xlsx

In [3]:
cell = tab.filter('Substance')
cell.assert_one()
observations = tab.filter('n').fill(DOWN).is_not_blank().is_not_whitespace()
Substance = cell.expand(DOWN).is_not_blank().is_not_whitespace()
Clients = cell.expand(RIGHT).is_not_blank().is_not_whitespace()
Dimensions = [
            HDim(Substance,'Treatment group',DIRECTLY,LEFT),
            HDim(Clients,'Clients in treatment',CLOSEST,LEFT),
            HDimConst('Measure Type','Count'),
            HDimConst('Unit','clients-in-treatment'),
            HDimConst('Period', 'gregorian-interval/2017-04-01T00:00:00/P1Y')
            ]

In [4]:
c1 = ConversionSegment(observations, Dimensions, processTIMEUNIT=True)
if is_interactive():
    savepreviewhtml(c1)

In [5]:
new_table = c1.topandas()

In [6]:
import numpy as np
new_table['OBS'].replace('', np.nan, inplace=True)
new_table.dropna(subset=['OBS'], inplace=True)
new_table.rename(columns={'OBS': 'Value'}, inplace=True)
new_table['Value'] = new_table['Value'].astype(int)
new_table['Value'] = new_table['Value'].map(lambda x:''
                                  if (x == '-')
                                  else int(x))

In [7]:
new_table['Treatment group'] = new_table['Treatment group'].str.rstrip('* ')

In [8]:
new_table['Treatment group'].unique()

array(['Opiate (not crack cocaine)', 'Both opiate and crack cocaine\xa0',
       'Crack cocaine (not opiate)', 'Cannabis', 'Cocaine',
       'Benzodiazepine', 'Amphetamine (other than ecstasy)', 'Other drug',
       'Hallucinogen', 'Other prescription drug', 'Anti-depressant',
       'Solvent', 'Major tranquiliser', 'Barbiturate', 'Alcohol',
       'Total number of individuals'], dtype=object)

In [9]:
new_table['Basis of treatment'] = new_table['Treatment group'].map(
    lambda x: {
        'Opiate (not crack cocaine)' : 'substance/opiate-not-crack-cocaine', 
        'Both opiate and crack cocaine\xa0' : 'substance/both-opiate-and-crack-cocaine',
       'Crack cocaine (not opiate)' : 'substance/crack-cocaine-not-opiate', 
        'Cannabis': 'substance/cannabis', 
        'Cocaine' :'substance/cocaine',
       'Benzodiazepine' :'substance/benzodiazepine', 
        'Amphetamine (other than ecstasy)' : 'substance/amphetamine-other-than-ecstasy', 
        'Other drug' : 'substance/other-drug',
       'Hallucinogen' : 'substance/hallucinogen', 
        'Other prescription drug' :'substance/other-prescription-drug', 
        'Anti-depressant' : 'substance/anti-depressant',
       'Solvent' : 'substance/solvent', 'Major tranquiliser' :'substance/major-tranquiliser', 
        'Barbiturate' : 'substance/barbiturate', 
        'Alcohol' : 'substance/alcohol',
       'Total number of individuals' : 'substance/total-number-of-individuals'
                
        }.get(x, x))

In [10]:
import urllib.request as request
import csv
import io
import requests
vrl="https://raw.githubusercontent.com/ONS-OpenData/ref_alcohol/master/codelists/substance-type.csv"
t=requests.get(vrl).content
g=pd.read_csv(io.StringIO(t.decode('utf-8')))
new_table = pd.merge(new_table, g, how = 'left', left_on = 'Clients in treatment', right_on = 'Label')
new_table.columns = ['Substance type' if x=='Notation' else x for x in new_table.columns]
new_table = new_table[['Period','Basis of treatment','Substance type','Measure Type','Value','Unit']]

In [11]:
new_table

,Period,Basis of treatment,Substance type,Measure Type,Value,Unit
0,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/opiate-not-crack-cocaine,opiate,Count,77248,clients-in-treatment
1,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/opiate-not-crack-cocaine,non-opiate-only,Count,0,clients-in-treatment
2,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/opiate-not-crack-cocaine,non-opiate-and-alcohol,Count,0,clients-in-treatment
3,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/opiate-not-crack-cocaine,alcohol-only,Count,0,clients-in-treatment
4,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/opiate-not-crack-cocaine,total,Count,77248,clients-in-treatment
5,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/both-opiate-and-crack-cocaine,opiate,Count,63941,clients-in-treatment
6,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/both-opiate-and-crack-cocaine,non-opiate-only,Count,0,clients-in-treatment
7,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/both-opiate-and-crack-cocaine,non-opiate-and-alcohol,Count,0,clients-in-treatment
8,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/both-opiate-and-crack-cocaine,alcohol-only,Count,0,clients-in-treatment
9,gregorian-interval/2017-04-01T00:00:00/P1Y,substance/both-opiate-and-crack-cocaine,total,Count,63941,clients-in-treatment
